# Modelo de Classificação


Nesse notebook faremos analises referentes a modelos preditivos de classificação, onde nossa variável alvo será FTR (full time result)

## Entendendo as variáveis do dataset

## 🏟️ Informações da Partida
* **Div** — Liga / divisão da partida.  
* **Date** — Data da partida (dd/mm/yy).  
* **Time** — Horário do início do jogo.  
* **HomeTeam** — Time mandante.  
* **AwayTeam** — Time visitante.  
* **Referee** — Árbitro da partida.

---

## ⚽ Placar e Resultados

* **FTHG** — Gols do mandante no fim do jogo.  
* **FTAG** — Gols do visitante no fim do jogo.  
* **FTR / Res** — Resultado final:  
  * `H` — vitória do mandante  
  * `D` — empate  
  * `A` — vitória do visitante  

* **HTHG** — Gols do mandante no intervalo.  
* **HTAG** — Gols do visitante no intervalo.  
* **HTR** — Resultado do intervalo (`H`, `D`, `A`).

---

## 📊 Estatísticas do Jogo

* **HS** — Finalizações do mandante.  
* **AS** — Finalizações do visitante.  

* **HST** — Finalizações no alvo do mandante.  
* **AST** — Finalizações no alvo do visitante.  

* **HC** — Escanteios do mandante.  
* **AC** — Escanteios do visitante.  

* **HF** — Faltas cometidas pelo mandante.  
* **AF** — Faltas cometidas pelo visitante.  

* **HY** — Cartões amarelos do mandante.  
* **AY** — Cartões amarelos do visitante.  

* **HR** — Cartões vermelhos do mandante.  
* **AR** — Cartões vermelhos do visitante.

---

## 💸 Odds – Bet365

* **B365H** — Odd vitória mandante.  
* **B365D** — Odd empate.  
* **B365A** — Odd vitória visitante.

---

## 💸 Odds – Bet & Win

* **BWH** — Bet&Win odd home.  
* **BWD** — Bet&Win odd draw.  
* **BWA** — Bet&Win odd away.

---

## 💸 Odds – Interwetten

* **IWH** — Interwetten odd home.  
* **IWD** — Interwetten odd draw.  
* **IWA** — Interwetten odd away.

---

## 💸 Odds – Pinnacle

* **PSH** — Pinnacle odd home.  
* **PSD** — Pinnacle odd draw.  
* **PSA** — Pinnacle odd away.

---

## 💸 Odds – VC Bet (inclui BetVictor)

* **VCH** — VC Bet home.  
* **VCD** — VC Bet draw.  
* **VCA** — VC Bet away.

---

## 💸 Odds – William Hill

* **WHH** — William Hill home.  
* **WHD** — William Hill draw.  
* **WHA** — William Hill away.

---

## 💸 Odds – Mercado (máximas)

* **MaxH** — Maior odd home no mercado.  
* **MaxD** — Maior odd draw.  
* **MaxA** — Maior odd away.

---

## 💸 Odds – Mercado (médias)

* **AvgH** — Média de odds home.  
* **AvgD** — Média de odds draw.  
* **AvgA** — Média de odds away.

---

## 🔢 Linhas de Gols – Over/Under

* **B365>2.5** — Bet365 over 2.5 gols.  
* **B365<2.5** — Bet365 under 2.5 gols.

* **P>2.5** — Pinnacle over 2.5 gols.  
* **P<2.5** — Pinnacle under 2.5 gols.

* **Max>2.5** — Maior odd over 2.5.  
* **Max<2.5** — Maior odd under 2.5.

* **Avg>2.5** — Média das odds over 2.5.  
* **Avg<2.5** — Média das odds under 2.5.

---

## 🧮 Asian Handicap (AH)

* **AHh** — Handicap do mandante (market side, desde 2019/20).  

* **B365AHH** — Bet365 AH home odds.  
* **B365AHA** — Bet365 AH away odds.

* **PAHH** — Pinnacle AH home odds.  
* **PAHA** — Pinnacle AH away odds.

* **MaxAHH** — Maior odd AH mandante no mercado.  
* **MaxAHA** — Maior odd AH visitante no mercado.

* **AvgAHH** — Média AH home no mercado.  
* **AvgAHA** — Média AH away no mercado.

## 🥗 Adicionais
* **Home_Streak** - Sequencia de vitórias (positivo) e derrotas (negativo)  
* **Away_Streak** - Sequencia de vitórias (positivo) e derrotas (negativo)  
* **Home_CumPoints** - Pontos acumulados na temporada mandante  
* **Away_CumPoints** - Pontos acumulados na temporada visitante  
* **Home_WinRate5** - Taxa de vitória nos últimos 5 jogos mandante  
* **Away_WinRate5** - Taxa de vitória nos últimos 5 jogos visitante  
* **Home_GFGA5** - Taxa de Gols Feitos por Gols sofridos mandante  
* **Away_GFGA5** - Taxa de Gols Feitos por Gols sofridos visitante  
* **Diff_CumPoints** - Diferença dos pontos acumulados  
* **B365_gap** - Diferença entre as odds (visitante - mandante) da bet365  
* **PS_gap** - Diferença entre as odds (visitante - mandante) da pinnacle  
* **PSH_prob_norm** - Odd convertida em probabilidade - prob vitória mandante pinnacle  
* **B365H_prob_norm** - Odd convertida em probabilidade - prob vitória mandante b365


# Importar Bibliotecas 

In [14]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

pd.set_option('display.max_columns', None)

# File Paths 

In [3]:
RAW_DATA_PATH = r'..\data\premier_completo_19_25.csv'
EXPANDED_DATA_PATH = r'..\data\premier_completo_expandido_19_25.csv'

# Carregar Dados 

In [10]:
df_raw = pd.read_csv(RAW_DATA_PATH)
print("\nPrimeiras linhas do conjunto de dados:")
df_raw.head(10)


Primeiras linhas do conjunto de dados:


,Div,Season,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,PSH,PSD,PSA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA
0,E0,19,09/08/2019,20:00,Liverpool,Norwich,4,1,H,4,0,H,M Oliver,15,12,7,5,9,9,11,2,0,2,0,0,1.14,10.00,19.00,1.15,9.59,18.05,1.16,10.00,23.00,1.14,8.75,19.83,1.40,3.00,1.40,3.11,1.45,3.11,1.41,2.92,1.96,1.94,1.97,1.95,1.97,2.00,1.94,1.94
1,E0,19,10/08/2019,12:30,West Ham,Man City,0,5,A,0,1,A,M Dean,5,14,3,9,6,13,1,1,2,2,0,0,12.00,6.50,1.22,11.68,6.53,1.26,13.00,6.75,1.29,11.84,6.28,1.25,1.44,2.75,1.49,2.77,1.51,2.77,1.48,2.65,2.00,1.90,2.02,1.90,2.02,1.92,1.99,1.89
2,E0,19,10/08/2019,15:00,Bournemouth,Sheffield United,1,1,D,0,0,D,K Friend,13,8,3,3,10,19,3,4,2,1,0,0,1.95,3.60,3.60,2.04,3.57,3.90,2.06,3.65,4.00,2.01,3.53,3.83,1.90,1.90,1.96,1.96,2.00,1.99,1.90,1.93,2.01,1.89,2.04,1.88,2.04,1.91,2.00,1.88
3,E0,19,10/08/2019,15:00,Burnley,Southampton,3,0,H,0,0,D,G Scott,10,11,4,3,6,12,2,7,0,0,0,0,2.62,3.20,2.75,2.71,3.31,2.81,2.80,3.33,2.85,2.68,3.22,2.78,2.10,1.72,2.17,1.77,2.20,1.78,2.12,1.73,1.92,1.98,1.93,2.00,1.94,2.00,1.91,1.98
4,E0,19,10/08/2019,15:00,Crystal Palace,Everton,0,0,D,0,0,D,J Moss,6,10,2,3,16,14,6,2,2,1,0,1,3.00,3.25,2.37,3.21,3.37,2.39,3.21,3.40,2.52,3.13,3.27,2.40,2.20,1.66,2.23,1.74,2.25,1.74,2.18,1.70,1.85,2.05,1.88,2.05,1.88,2.09,1.84,2.04
5,E0,19,10/08/2019,15:00,Watford,Brighton,0,3,A,0,1,A,C Pawson,11,5,3,3,15,11,5,2,0,1,0,0,1.90,3.40,4.00,1.98,3.44,4.37,2.00,3.50,4.60,1.94,3.41,4.26,2.10,1.72,2.19,1.76,2.24,1.76,2.16,1.71,1.95,1.95,1.98,1.95,1.98,1.98,1.94,1.94
6,E0,19,10/08/2019,17:30,Tottenham,Aston Villa,3,1,H,0,1,A,C Kavanagh,31,7,7,4,13,9,14,0,1,0,0,0,1.30,5.25,10.00,1.30,5.84,10.96,1.33,5.95,12.00,1.30,5.53,10.51,1.66,2.20,1.64,2.40,1.70,2.40,1.65,2.26,1.97,1.93,1.99,1.93,2.00,2.00,1.93,1.94
7,E0,19,11/08/2019,14:00,Leicester,Wolves,0,0,D,0,0,D,A Marriner,15,8,1,2,3,13,12,3,0,2,0,0,2.20,3.20,3.40,2.21,3.34,3.66,2.29,3.38,3.66,2.22,3.28,3.48,2.20,1.66,2.23,1.74,2.25,1.74,2.17,1.70,1.90,2.00,1.90,2.04,1.95,2.04,1.91,1.98
8,E0,19,11/08/2019,14:00,Newcastle,Arsenal,0,1,A,0,0,D,M Atkinson,9,8,2,2,12,7,5,3,1,3,0,0,4.50,3.75,1.72,4.58,3.93,1.81,4.70,4.00,1.83,4.49,3.82,1.79,1.80,2.00,1.83,2.10,1.83,2.14,1.77,2.07,1.85,2.05,1.86,2.07,1.88,2.08,1.85,2.03
9,E0,19,11/08/2019,16:30,Man United,Chelsea,4,0,H,1,0,H,A Taylor,11,18,5,7,15,13,3,5,3,4,0,0,2.10,3.30,3.50,2.21,3.37,3.63,2.28,3.43,3.63,2.19,3.32,3.49,2.00,1.80,2.05,1.87,2.10,1.87,2.01,1.83,1.90,2.00,1.90,2.04,1.92,2.04,1.89,2.00


In [15]:
df_raw_expanded = pd.read_csv(EXPANDED_DATA_PATH)
print("\nPrimeiras linhas do conjunto de dados:")
df_raw_expanded.sample(10)



Primeiras linhas do conjunto de dados:


,Div,Season,Date,Time,MatchID,HomeTeam,Home_CumPoints,Home_Streak,AwayTeam,Away_Streak,Away_CumPoints,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,PSH,PSD,PSA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,Goal_Diff_Home,Home_Win,B365H_prob,B365D_prob,B365A_prob,PSH_prob,PSA_prob,PSD_prob,B365H_prob_norm,B365D_prob_norm,B365A_prob_norm,PSH_prob_norm,PSA_prob_norm,PSD_prob_norm,Points_Home,Points_Away,B365_gap,PS_gap,Home_WinRate5,Home_GF5,Home_GA5,Home_GFGA5,Away_WinRate5,Away_GF5,Away_GA5,Away_GFGA5
653,E0,20,2021-03-08,18:00,653,Chelsea,47,1,Everton,3,46,2,0,H,1,0,H,D Coote,19,7,9,1,11,12,7,3,0,3,0,0,1.53,4.00,6.50,1.55,4.33,6.72,1.62,4.40,6.90,1.55,4.14,6.29,1.98,1.92,1.98,1.93,2.00,1.98,1.93,1.89,1.98,1.95,1.95,1.97,2.03,2.00,1.97,1.91,2,1,0.653595,0.250000,0.153846,0.645161,0.148810,0.230947,0.618091,0.236420,0.145489,0.629476,0.145192,0.225332,3,0,-4.97,-5.17,0.6,6.0,2.0,3.000000,0.600000,5.0,5.0,1.000000
2381,E1,24,2025-04-08,19:45,2381,Bristol City,60,1,West Brom,-2,57,2,1,H,0,0,D,M Donohue,12,13,7,5,8,14,4,5,1,1,0,1,2.40,3.40,2.87,2.48,3.42,2.96,2.51,3.42,3.00,2.43,3.33,2.89,2.10,1.73,2.15,1.76,2.16,1.79,2.10,1.73,2.11,1.79,2.14,1.79,2.14,1.83,2.08,1.77,1,1,0.416667,0.294118,0.348432,0.403226,0.337838,0.292398,0.393373,0.277675,0.328953,0.390170,0.326899,0.282930,3,0,-0.47,-0.48,0.4,6.0,5.0,1.200000,0.200000,3.0,4.0,0.750000
1535,E0,23,2023-08-19,15:00,1535,Wolves,0,-1,Brighton,1,3,1,4,A,0,1,A,A Madley,16,16,5,8,14,11,5,3,3,6,1,0,4.00,3.80,1.85,4.12,3.92,1.88,4.25,4.00,1.97,4.00,3.81,1.89,1.73,2.10,1.75,2.17,1.80,2.18,1.74,2.12,2.03,1.87,2.03,1.88,2.05,1.91,1.99,1.87,-3,0,0.250000,0.263158,0.540541,0.242718,0.531915,0.255102,0.237260,0.249747,0.512994,0.235710,0.516555,0.247736,0,3,2.15,2.24,0.0,0.0,1.0,0.000000,1.000000,4.0,1.0,4.000000
836,E0,21,2021-10-16,12:30,836,Watford,7,-1,Liverpool,0,15,0,5,A,0,2,A,J Moss,6,19,2,8,4,10,1,9,0,0,0,0,8.50,5.50,1.33,9.43,5.49,1.36,9.50,5.75,1.39,8.67,5.33,1.36,1.57,2.37,1.59,2.50,1.65,2.51,1.58,2.43,1.86,2.07,1.88,2.05,1.89,2.09,1.85,2.02,-5,0,0.117647,0.181818,0.751880,0.106045,0.735294,0.182149,0.111901,0.172939,0.715160,0.103611,0.718420,0.177969,0,3,7.17,8.07,0.2,4.0,6.0,0.666667,0.400000,12.0,6.0,2.000000
1002,E0,21,2022-02-20,14:00,1002,Leeds,23,-1,Man United,1,43,2,4,A,0,2,A,P Tierney,16,15,6,9,13,19,3,3,6,3,0,0,3.90,3.80,1.85,4.12,3.84,1.93,4.20,4.10,1.95,3.91,3.86,1.90,1.61,2.30,1.65,2.38,1.70,2.40,1.63,2.30,1.99,1.91,2.00,1.93,2.01,1.95,1.97,1.91,-2,0,0.256410,0.263158,0.540541,0.242718,0.518135,0.260417,0.241872,0.248237,0.509892,0.237663,0.507344,0.254993,0,3,2.05,2.19,0.4,9.0,10.0,0.900000,0.600000,8.0,3.0,2.666667
577,E0,20,2021-01-30,12:30,577,Everton,33,0,Newcastle,-5,19,0,2,A,0,0,D,S Attwell,11,13,3,3,13,6,7,12,3,4,0,0,1.55,4.00,6.50,1.61,4.03,6.49,1.63,4.18,6.70,1.59,3.95,6.27,2.00,1.80,2.05,1.86,2.06,1.90,1.99,1.84,2.07,1.83,2.10,1.84,2.10,1.91,2.05,1.83,-2,0,0.645161,0.250000,0.153846,0.621118,0.154083,0.248139,0.615021,0.238321,0.146659,0.606952,0.150569,0.242479,0,3,-4.95,-4.88,0.6,6.0,4.0,1.500000,0.000000,2.0,10.0,0.200000
788,E0,21,2021-08-29,14:00,788,Tottenham,6,2,Watford,-1,3,1,0,H,1,0,H,A Marriner,15,9,8,2,8,14,10,3,3,3,0,0,1.36,4.75,9.00,1.45,4.42,9.04,1.48,4.75,9.10,1.44,4.48,8.42,1.80,2.00,1.88,2.02,1.91,2.05,1.85,2.00,2.08,1.85,2.10,1.84,2.10,1.90,2.04,1.85,1,1,0.735294,0.210526,0.111111,0.689655,0.110619,0.226244,0.695688,0.199186,0.105126,0.671839,0.107762,0.220400,3,0,-7.64,-7.59,1.0,2.0,0.0,NaN,0.500000,3.0,4.0,0.750000
327,E0,19,2020-07-05,14:15,327,Newcastle,42,1,West Ham,1,30,2,2,D,1,1,D,C Pawson,11,17,4,7,9,11,6,4,0,0,0,0,2.60,3.30,2.75,2.72,3.31,2.77,2.77,3.48,2.90,2.66,3.28,2.74,2.00,1.80,2.05,1.86,2.09,1.91,2.00,1.82,1.92,1.98,1.94,1.98,1.99,2.01,1.91,1.96,0,0,0.384615,0.303030,0.363636,0.367647,0.361011,0.302115,0.365854,0.288248,0.345898,0.356671,0.350233,0.293095,1,1,-0.15,-0.05,0.6,9.0,2.0